In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from test_dataset.german_credit import GermanCreditDataset

#### 0. Pick the data to be factual("Risk"=1)

In [3]:
dataset = GermanCreditDataset()
df = dataset.get_dataframe()
df.head(3)

d:\Github-desktop\COLA\test_dataset\german_credit.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.target = self.df[self.target_name].replace({"good": 0, "bad": 1})


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1
2,49,1,1,1,1,0,2096,12,3,0


In [4]:
df_Risk_1 = df[df['Risk'] == 1]
df_Risk_1 = df_Risk_1.sample(5)
df_Risk_1.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
166,33,0,2,1,1,1,1131,18,4,1
155,20,0,2,2,1,1,1282,12,4,1
237,61,1,1,2,2,2,2767,21,0,1
265,37,1,2,1,1,2,802,15,5,1
531,28,0,2,2,2,2,2631,15,1,1


In [5]:
df_without_target = df_Risk_1.drop(columns=['Risk']).copy()
df_without_target.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
166,33,0,2,1,1,1,1131,18,4
155,20,0,2,2,1,1,1282,12,4
237,61,1,1,2,2,2,2767,21,0
265,37,1,2,1,1,2,802,15,5
531,28,0,2,2,2,2,2631,15,1


In [6]:
feature_names = df_without_target.columns
feature_names

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose'],
      dtype='object')

#### 1. Initialize data interface

In [7]:
from xai_cola import data_interface 
data1 = data_interface.PandasData(df_without_target, target_name='Risk')

#### 2. Initialize model interface

In [8]:
import joblib
lgbmcClassifier = joblib.load('lgbm_GremanCredit.pkl')
print(f'----lgbm_GremanCredit.pkl model has been loaded----')

----lgbm_GremanCredit.pkl model has been loaded----


In [9]:
from xai_cola import ml_model_interface
ml_model1 = ml_model_interface.Model(model=lgbmcClassifier, backend="sklearn")

#### 3.Choose the CounterfactualExplanation Algorithm

In [10]:
from counterfactual_explainer import DiCE
explainer = DiCE(ml_model=ml_model1)

In [30]:
factual, counterfactual = explainer.generate_counterfactuals(data=data1,
                                                             factual_class=1,
                                                             total_cfs=2)

100%|██████████| 5/5 [00:00<00:00, 10.17it/s]


In [31]:
factual

array([[  33,    0,    2,    1,    1,    1, 1131,   18,    4],
       [  20,    0,    2,    2,    1,    1, 1282,   12,    4],
       [  61,    1,    1,    2,    2,    2, 2767,   21,    0],
       [  37,    1,    2,    1,    1,    2,  802,   15,    5],
       [  28,    0,    2,    2,    2,    2, 2631,   15,    1]],
      dtype=int64)

In [32]:
counterfactual

array([[  48,    0,    2,    1,    1,    1, 2320,   18,    4],
       [  58,    0,    2,    1,    1,    1, 1131,   18,    4],
       [  49,    0,    2,    2,    1,    1, 1282,   18,    4],
       [  20,    0,    2,    2,    2,    1, 2366,   12,    4],
       [  39,    1,    1,    2,    2,    2, 2767,   21,    0],
       [  61,    1,    1,    2,    2,    2, 1522,   12,    0],
       [  37,    1,    2,    1,    1,    2, 1613,   15,    2],
       [  37,    1,    2,    1,    1,    2,  802,   13,    5],
       [  60,    0,    2,    2,    2,    2, 2631,   21,    1],
       [  28,    0,    2,    2,    2,    1, 1946,   15,    1]],
      dtype=int64)

#### 4. Choose policy and make limitation

In [33]:
from xai_cola.counterfactual_limited_actions import COLA
refiner = COLA(
    data=data1,
    ml_model=ml_model1,
    x_factual=factual,
    x_counterfactual=counterfactual,
)
refiner.set_policy(
    matcher="ot",
    attributor="pshap",
    Avalues_method="max"
    )

You choose the Policy: pshap With Optimal Transport Matching, Avalues_method is max


In [40]:
factual, ce, ace = refiner.get_refined_counterfactual(limited_actions=10)

INFO:shap:num_full_subsets = 4
INFO:shap:phi = [ 0.39598783  0.01972025  0.07610815 -0.05676232 -0.0098155   0.0441457
  0.10828504  0.0748535  -0.03210984]
INFO:shap:num_full_subsets = 4
INFO:shap:phi = [-0.0650176   0.03371465  0.03383595 -0.02292082 -0.02561062  0.59846585
  0.06963699 -0.09473767]
INFO:shap:num_full_subsets = 2
INFO:shap:phi = [ 0.01980302 -0.02183041 -0.03110776 -0.00612183  0.0221252 ]
INFO:shap:num_full_subsets = 3
INFO:shap:phi = [ 0.17910963 -0.00834012 -0.04961627 -0.04563258 -0.0824306  -0.00647138]
INFO:shap:num_full_subsets = 3
INFO:shap:phi = [ 0.3151353   0.08900187  0.02275615  0.11547869  0.00850062 -0.17495225
  0.13741639]


In [50]:
import pandas as pd
pd.DataFrame(refiner._get_data_composer())

,0,1,2,3,4,5,6,7,8
0,49.0,0.0,2.0,2.0,1.0,1.0,1282.0,18.0,4.0
1,37.0,1.0,2.0,1.0,1.0,2.0,1613.0,15.0,2.0
2,39.0,1.0,1.0,2.0,2.0,2.0,2767.0,21.0,0.0
3,58.0,0.0,2.0,1.0,1.0,1.0,1131.0,18.0,4.0
4,48.0,0.0,2.0,1.0,1.0,1.0,2320.0,18.0,4.0


In [41]:
factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,33,0,2,1,1,1,1131,18,4,1
1,20,0,2,2,1,1,1282,12,4,1
2,61,1,1,2,2,2,2767,21,0,0
3,37,1,2,1,1,2,802,15,5,0
4,28,0,2,2,2,2,2631,15,1,1


In [44]:
ce

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,48,0,2,1,1,1,2320,18,4,0
1,58,0,2,1,1,1,1131,18,4,0
2,49,0,2,2,1,1,1282,18,4,0
3,20,0,2,2,2,1,2366,12,4,0
4,39,1,1,2,2,2,2767,21,0,0
5,61,1,1,2,2,2,1522,12,0,0
6,37,1,2,1,1,2,1613,15,2,0
7,37,1,2,1,1,2,802,13,5,0
8,60,0,2,2,2,2,2631,21,1,0
9,28,0,2,2,2,1,1946,15,1,0


In [43]:
ace

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,49,0,2,1,1,1,1282,18,4,0
1,20,0,2,2,1,1,1613,12,2,0
2,61,1,1,2,2,2,2767,21,0,0
3,58,1,2,1,1,2,802,15,5,0
4,48,0,2,1,2,1,2631,18,4,0
